# RAG Based on ReRanker
- Hybrid Search
- ReRanker

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fd12bc720b0>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fd169b79e40>])

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    
    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

## 4. Invocation (prediction)

In [13]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7fd118487b20>


In [14]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-04-37-45-120" # ml.m5.2xlarge
endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge
deserializer = "application/json"

In [15]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [16]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.999653697013855}, {'label': 'LABEL_0', 'score': 0.025255175307393074}]
CPU times: user 19.8 ms, sys: 1.13 ms, total: 20.9 ms
Wall time: 92.7 ms


## 5. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [17]:
from utils.proc_docs import get_parameter

In [18]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

In [19]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [20]:
index_name = "v13-genai-poc-knox-titan-eval-sample-0.05-1024c-256o"

### OpenSearch Client 생성

In [21]:
from utils.opensearch import opensearch_utils

In [22]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### LangChain OpenSearch VectorStore 생성

In [23]:
from langchain.vectorstores import OpenSearchVectorSearch

In [24]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

## 6. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [25]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [26]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    vector_db=vector_db,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    
    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

### Retrieval example
- default search

In [27]:
from utils.rag import show_context_used

In [28]:
query = "knox"

In [29]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: knox
2. # documents: 5
3. Documents: 

-----------------------------------------------
1. Chunk: 674 Characters
-----------------------------------------------
What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better
manage your fleet of devices. It includes individual services such as Knox Platform for Enterprise,
Knox Mobile Enrollment, Knox Manage, Knox E-FOTA, and Knox Asset Intelligence. Key features of Knox
Suite include: Secure - Ensure your business data is protected with managed security features at
your control. Deploy - Enroll corporate devices to EMM automatically and securely, right out of the
box. Manage - Enable comprehensive management over corporate devices and OS versions. Analyze -
Strengthen in-depth device analytics for mobile productivity and asset management.
metadata:
 {'source': 'all_processed_data.json', 'seq_num': 74, 'title': ' What is Knox Suite?', 'url':
'https://docs.

- update parameters

In [37]:
opensearch_hybrid_retriever.update_search_params(
    k=3,
    minimum_should_match=30,
    filter=[
        {"term": {"metadata.seq_num": "64"}},
    ],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
)

In [38]:
query = "knox"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: knox
2. # documents: 3
3. Documents: 

-----------------------------------------------
1. Chunk: 997 Characters
-----------------------------------------------
Learn more about Knox Suite. This section describes where you can find instructions on how to
perform tasks using the different products included in Knox Suite. General documentation for the
Knox Suite services Knox Service , See - , - Knox Manage , Knox Manage Admin Guide Knox Platform for
Enterprise , Knox Platform for Enterprise Admin Guide Knox Mobile Enrollment , Knox Mobile
Enrollment Admin Guide Knox E-FOTA , Knox E-FOTA Admin Guide Knox Asset Intelligence , Knox Asset
Intelligence Admin Guide Knox Capture , Knox Capture Admin Guide Knox Authentication Manager , Knox
Authentication Manager Admin Guide Knox Suite operations For information on See Knox service How to
set up Knox Manage with Knox Suite Workflow:"Set up Knox Manage with a Knox Suite license Knox
Manage How to che

## 5. RAG using RetrievalQA powered by LangChain

In [39]:
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [40]:
prompt_template = """
\n\nHuman: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "Could not find answer in given contexts."

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### Update Search Params (Optional)

In [41]:
from langchain.chains import RetrievalQA

In [46]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KPE"}},
    #],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    verbose=False
)

### Request

In [47]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [48]:
query = "knox"
response = qa(query)

 Based on the given context, here are the key points about Knox Suite:

- Knox Suite is a bundled offering of Knox solutions for enterprises to help manage and secure devices. It includes Knox Platform for Enterprise, Knox Mobile Enrollment, Knox Manage, Knox E-FOTA, and Knox Asset Intelligence.

- Knox Mobile Enrollment allows bulk enrollment of devices into an EMM. With a Knox Suite license, it provides advanced profiles for more control over locking devices. 

- Knox Manage is an EMM that allows remote management of devices through policies to control usage. 

- Knox E-FOTA enables remote deployment of OS and security updates without user interaction.

- Knox Asset Intelligence provides analytics for monitoring and managing devices.

- Knox Suite license allows using all included Knox products on devices with a single key.

- Benefits include an end-to-end solution covering enterprise mobility, and military-grade security for devices.

- Documentation on how to use different Knox se

In [49]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])

##################################
query:  knox
##################################


### Answer ###
 Based on the given context, here are the key points about Knox Suite:

- Knox Suite is a bundled offering of Knox solutions for enterprises to help manage and secure
devices. It includes Knox Platform for Enterprise, Knox Mobile Enrollment, Knox Manage, Knox E-FOTA,
and Knox Asset Intelligence.

- Knox Mobile Enrollment allows bulk enrollment of devices into an EMM. With a Knox Suite license,
it provides advanced profiles for more control over locking devices.

- Knox Manage is an EMM that allows remote management of devices through policies to control usage.

- Knox E-FOTA enables remote deployment of OS and security updates without user interaction.

- Knox Asset Intelligence provides analytics for monitoring and managing devices.

- Knox Suite license allows using all included Knox products on devices with a single key.

- Benefits include an end-to-end solution covering enterprise m

In [ ]:
#!pip install -U FlagEmbedding
%%time
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=False) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores)